In [2]:
from model.mhformer import Model
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ast
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
from pathlib import Path
import json
import numpy as np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

N_KPTS = 133
N_FRAMES = 60

cuda:0


In [4]:
class Args:
    def __init__(self):
        self.frames = 60
        self.n_joints = 133
        self.channel = 256
        self.out_joints = 133
        self.layers = 4
        self.d_hid = 512

args = Args()
model = Model(args).to(device)

In [5]:
model.load_state_dict(torch.load("model/mhformer.pth", weights_only=True))
model.eval()

Model(
  (norm_1): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
  (norm_2): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
  (norm_3): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
  (Transformer_encoder_1): Transformer(
    (pos_drop): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((60,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=60, out_features=180, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=60, out_features=60, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((60,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=60, out_features=120, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=120, out_features=60, bias=True)
          (drop

In [7]:
with open("IMG_8212.json", "r") as file:
    data = json.load(file)

data = data["instance_info"][0]["keypoints"]

In [14]:
keypoints_2d = torch.tensor(data, dtype=torch.float, device=device)
keypoints_2d = keypoints_2d.unsqueeze(0)
keypoints_2d = keypoints_2d.repeat((60, 1, 1))
keypoints_2d = keypoints_2d.unsqueeze(0)
keypoints_3d = model(keypoints_2d)
keypoints_3d.shape

torch.Size([1, 60, 133, 3])

In [18]:
result = keypoints_3d[0, 0].detach().cpu().numpy()
np.savez("keypoints", result)

In [19]:
keypoints_2d

tensor([[[[2493.8547, 1714.1366],
          [2609.0972, 1611.3046],
          [2482.2881, 1633.0760],
          ...,
          [2091.5415, 3358.7749],
          [2090.1392, 3358.8303],
          [2092.8386, 3358.3726]],

         [[2493.8547, 1714.1366],
          [2609.0972, 1611.3046],
          [2482.2881, 1633.0760],
          ...,
          [2091.5415, 3358.7749],
          [2090.1392, 3358.8303],
          [2092.8386, 3358.3726]],

         [[2493.8547, 1714.1366],
          [2609.0972, 1611.3046],
          [2482.2881, 1633.0760],
          ...,
          [2091.5415, 3358.7749],
          [2090.1392, 3358.8303],
          [2092.8386, 3358.3726]],

         ...,

         [[2493.8547, 1714.1366],
          [2609.0972, 1611.3046],
          [2482.2881, 1633.0760],
          ...,
          [2091.5415, 3358.7749],
          [2090.1392, 3358.8303],
          [2092.8386, 3358.3726]],

         [[2493.8547, 1714.1366],
          [2609.0972, 1611.3046],
          [2482.2881, 1633.0760],

In [28]:
temp = torch.cat((keypoints_2d, torch.zeros((1, 60, 133, 1), device=device)), dim=-1)
np.savez('idk', temp[0, 0].cpu().numpy(), keypoints_3d[0, 0].cpu().detach().numpy())

In [20]:
keypoints_3d

tensor([[[[ -32.4070, -810.9669, -304.9438],
          [  13.6878, -808.4675, -335.5195],
          [ -45.7688, -834.4612, -236.6948],
          ...,
          [-335.9191,  228.8342,   41.5663],
          [-333.3473,  227.9351,   28.5436],
          [-327.6690,  218.1649,   27.0325]],

         [[ -44.4964, -790.9933, -336.1886],
          [   1.4576, -788.0034, -367.2071],
          [ -57.5554, -814.1517, -267.6473],
          ...,
          [-346.9630,  243.7602,   10.5157],
          [-344.5704,  242.7551,   -2.5796],
          [-339.1542,  232.9596,   -4.0230]],

         [[ -26.5347, -817.0125, -304.4930],
          [  19.4820, -814.5652, -335.2736],
          [ -39.7470, -840.5464, -236.1372],
          ...,
          [-328.8535,  224.8457,   43.2019],
          [-326.2545,  223.9070,   30.1350],
          [-320.5480,  214.1002,   28.6071]],

         ...,

         [[ -43.0672, -797.3382, -325.4843],
          [   2.8726, -794.5076, -356.1923],
          [ -56.3128, -820.5638, -